# **Weather AUS Model**


### 1- Import & Display dataset
***

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
pd.set_option('display.max_columns', 200)

from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

from pathlib import Path
import os

### 2- Load Data
***

In [ ]:
script_folder = Path(os.getcwd())
dataset_folder = script_folder / "data"

print("Code Location: ", script_folder)
print("Dataset Location: ", dataset_folder)

In [ ]:
df = pd.read_csv(dataset_folder / "weatherAUS.csv")
print(f"Original shape: {df.shape}")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

### 3- Visualization Functions
***

In [ ]:
def plot_bar(data, title, xlabel='', ylabel='Count', color='skyblue', figsize=(6,4), kind='bar'):
    plt.figure(figsize=figsize)
    data.plot(kind=kind, color=color)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(rotation=0 if kind == 'bar' else None)
    plt.show()

In [ ]:
def plot_correlations(df, target_col, top_n=10, ascending=False, threshold=None):
    df_temp = df.copy()
    df_temp[target_col] = df_temp[target_col].map({'Yes': 1, 'No': 0})
    
    corr_data = (
        df_temp.select_dtypes(include=[np.number])
        .corr()[target_col]
        .drop(target_col)
        .dropna()
        .abs()
        .sort_values(ascending=ascending)
        .head(top_n)
    )
    
    if threshold:
        corr_data = corr_data.loc[lambda x: x < threshold]
    
    ax = corr_data.plot(kind='bar', color='seagreen', figsize=(10,5))
    
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x() + p.get_width()/2, height, f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)
    
    title = f'Top {top_n} Correlations' if not ascending else f'Least Correlations (<{threshold})'
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Feature')
    plt.ylabel('Correlation Coefficient')
    plt.show()

In [ ]:
def plot_boxplots(df, n_cols=4):
    numerical_cols = list(df.select_dtypes(include=np.number).columns)
    n_rows = math.ceil(len(numerical_cols) / n_cols)
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 4))
    axes = axes.flatten()
    
    for idx, col in enumerate(numerical_cols):
        sns.boxplot(x=df[col], ax=axes[idx])
        axes[idx].set_title(f'Boxplot of {col}', fontsize=12, fontweight='bold')
        axes[idx].set_xlabel('')
    
    for idx in range(len(numerical_cols), len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.show()

### 4- Initial EDA
***

In [ ]:
plot_bar(
    data=df['RainTomorrow'].value_counts(), 
    title='Target Variable Distribution',
    xlabel='Rain Tomorrow',
    color=['skyblue', 'coral'] #type:ignore
)

In [ ]:
missing_pct = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False).head(15)
plot_bar(
    data=missing_pct, 
    title='Top 15 Features by Missing Values (%)',
    xlabel='Missing Percentage',
    color='indianred',
    kind='barh'
)

In [ ]:
def plot_rainfall_distribution():
    plt.figure(figsize=(6,4))
    df['Rainfall'].plot(
        kind='hist', bins=50, color='steelblue', edgecolor='black'
    )
    plt.title('Rainfall Distribution', fontsize=14, fontweight='bold')
    plt.xlabel('Rainfall (mm)')
    plt.ylabel('Frequency')
    plt.show()
    
plot_rainfall_distribution()

In [ ]:
plot_correlations(
    df=df, 
    target_col='RainTomorrow', 
    top_n=10, 
    ascending=False
)

In [ ]:
plot_correlations(
    df=df, 
    target_col='RainTomorrow',
    top_n=11,
    ascending=True,
    threshold=0.25
)

### 5- Data Cleaning
***

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df = df.drop(columns=["Evaporation", "Sunshine"])

target_col = "RainTomorrow"
df = df.dropna(subset=[target_col])

In [ ]:
numerical_cols = list(df.select_dtypes(include=np.number).columns)
categorical_cols = list(df.select_dtypes(include='object').columns)
print(numerical_cols)
print(categorical_cols)

### 6- Feature Engineering
***

In [ ]:
print(df.shape)

In [ ]:
# create Year, Month, Day, DayOfWeek and dropped Date col
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.dayofweek
df = df.drop(columns=['Date'])

# Create cyclical features for temporal patterns
df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
df['Day_sin'] = np.sin(2 * np.pi * df['Day'] / 31)
df['Day_cos'] = np.cos(2 * np.pi * df['Day'] / 31)

df = df.drop(columns=['Temp9am', 'Temp3pm']) #TODO: remove or leave?

In [ ]:
print(df.shape)

In [ ]:
numerical_cols = list(df.select_dtypes(include=np.number).columns)
categorical_cols = list(df.select_dtypes(include='object').columns)
print(numerical_cols)
print(categorical_cols)

In [ ]:
plot_correlations(
    df=df, 
    target_col='RainTomorrow',
    top_n=25,
    ascending=True,
    threshold=0.25
)

### 7- Handle Missing Values
***

In [ ]:
numerical_cols = list(df.select_dtypes(include=np.number).columns)
categorical_cols = list(df.select_dtypes(include='object').columns)

date_cols = ['Year', 'Month', 'Day', 'DayOfWeek', 'Month_sin', 'Month_cos', 'Day_sin', 'Day_cos']
numerical_cols_not_date = [col for col in numerical_cols if col not in date_cols]

In [ ]:
for col in numerical_cols_not_date:
    max_consecutive_missing = df[col].isna().astype(int).groupby(
        df[col].notna().astype(int).cumsum()
    ).sum().max()
    
    if max_consecutive_missing < 7:
        df[col] = df[col].interpolate(method='nearest', limit_direction='both')
    else:
        monthly_means = df.groupby(['Year', 'Month'])[col].transform('mean')
        df.loc[:, col] = df[col].fillna(monthly_means)

print("Missing values after imputation:")
print(df[numerical_cols_not_date].isnull().sum())

### 8- Handling Outliers
***

In [ ]:
plot_boxplots(df)

In [ ]:
def remove_outliers_iqr(df):
    print("Old Shape:", df.shape)
    numerical_cols_current = list(df.select_dtypes(include=np.number).columns)
    for col in numerical_cols_current:
        # Use nanpercentile to handle NaN values correctly
        Q1 = np.nanpercentile(df[col], 25)
        Q3 = np.nanpercentile(df[col], 75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Create a boolean mask to filter outliers (keeps NaN values)
        mask = (df[col] >= lower_bound) & (df[col] <= upper_bound) | df[col].isna()
        df = df[mask]
        
    print("New Shape:", df.shape)
    return df

df = remove_outliers_iqr(df)

In [ ]:
plot_boxplots(df)

### 9- Data-Target Split
***

In [ ]:
X = df.drop(columns=[target_col])
y = df[target_col]

### 10- Build Preprocessing Pipeline
***

In [ ]:
print(y[5:10])

In [ ]:
le_target = LabelEncoder()
y = le_target.fit_transform(y)

In [ ]:
print(y[5:10]) # type: ignore

In [ ]:
numerical_cols = list(X.select_dtypes(include=np.number).columns)
categorical_cols = list(X.select_dtypes(include='object').columns)

In [ ]:
categorical_binary_cols = [col for col in categorical_cols if X[col].nunique() == 2]
categorical_multi_class_cols = [col for col in categorical_cols if X[col].nunique() > 2]

print(f"Numerical columns ({len(numerical_cols)}): {numerical_cols}")
print(f"Binary categorical ({len(categorical_binary_cols)}): {categorical_binary_cols}", )
print(f"Multi-class categorical ({len(categorical_multi_class_cols)}): {categorical_multi_class_cols}")

In [ ]:
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

multiclass_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('Numerical Preprocessing', numerical_transformer, numerical_cols),
    ('Categorical Binary Preprocessing', binary_transformer, categorical_binary_cols),
    ('Categorical Multi-Class Preprocessing', multiclass_transformer, categorical_multi_class_cols)
])

### 11- Train-Test Split
***

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("Train Records:",X_train.shape[0])
print("Test Records:",X_test.shape[0])

In [ ]:
# display the unprocessed training data
X_train_df = pd.DataFrame(X_train, columns=X.columns)
print(X_train_df.shape)
X_train_df.head()

X_test_df = pd.DataFrame(X_test, columns=X.columns)
print(X_test_df.shape)
X_train_df.head()

##### Visulize Pipeline results

In [ ]:
# model will not use this (just for demonstration)
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test) 

In [ ]:
X_train_processed_df = pd.DataFrame(X_train_processed, columns=preprocessor.get_feature_names_out()) #type:ignore
print(X_train_processed_df.shape)
X_test_processed_df = pd.DataFrame(X_test_processed, columns=preprocessor.get_feature_names_out()) #type:ignore
print(X_test_processed_df.shape)
X_train_processed_df.head()

### 12- Build Model Pipeline with GridSearch
***

In [ ]:
pca = PCA(n_components=0.95)  # Retain 95% variance

In [ ]:
models_dir = Path("Models")

def save_model(model, model_file: str):
    filepath = models_dir / model_file
    print("Model saved as ", model_file)
    joblib.dump(model, filepath)

def load_model(model_file: str):
    filepath = models_dir / model_file
    if filepath.exists():
        model = joblib.load(filepath)
        print(model_file, " Model loaded")
        return model
    print("Model not found")
    return None

#### Random Forest

In [ ]:
best_random_forest = load_model("best_random_forest.pkl")

rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', pca),
    ('classifier', RandomForestClassifier(random_state=42, n_jobs=8))
])

rf_param_grid = {
    'classifier__n_estimators': [400],
    'classifier__max_depth': [None, 10],
    'classifier__min_samples_split': [3, 5],
    'classifier__min_samples_leaf': [1, 2],
    'classifier__max_features': ['sqrt', 'log2']
}

rf_grid_search = GridSearchCV(
    rf_pipeline,
    rf_param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=1,
    verbose=3
)

if best_random_forest == None:
    rf_grid_search.fit(X_train, y_train)
    best_random_forest = rf_grid_search.best_estimator_
    save_model(best_random_forest, "best_random_forest.pkl")

In [ ]:
best_xgb_boost = load_model("best_xgb_boost.pkl")
xgb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', pca),
    ('classifier', XGBClassifier(random_state=42, n_jobs=8, eval_metric='logloss'))
])

xgb_param_grid = {
    'classifier__n_estimators': [200, 250],
    'classifier__max_depth': [6, 8],
    'classifier__learning_rate': [0.04, 0.06, 0.08],
    'classifier__subsample': [0.85, 0.9],
    'classifier__colsample_bytree': [0.8, 0.85],
    'classifier__gamma': [0.2, 0.25, 0.3],
    'classifier__min_child_weight': [1, 2]
}

xgb_grid_search = GridSearchCV(
    xgb_pipeline,
    xgb_param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=1,
    verbose=3
)

if best_xgb_boost == None:
    xgb_grid_search.fit(X_train, y_train)
    best_xgb_boost = xgb_grid_search.best_estimator_
    save_model(best_xgb_boost, "best_xgb_boost.pkl")
    

In [ ]:
best_mlp = load_model("best_mlp.pkl")

mlp_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', pca),
    ('classifier', MLPClassifier(random_state=42, max_iter=1000, early_stopping=True))
])

mlp_param_grid = {
    'classifier__hidden_layer_sizes': [(130, 89), (100, 50), (150, 100), (200,)],
    'classifier__activation': ['tanh', 'relu'],
    'classifier__alpha': [0.001, 0.005, 0.01],
    'classifier__learning_rate': ['adaptive', 'constant'],
    'classifier__solver': ['adam']
}

mlp_grid_search = GridSearchCV(
    mlp_pipeline,
    mlp_param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=1,
    verbose=3
)

if best_mlp == None:
    mlp_grid_search.fit(X_train, y_train)
    best_mlp = mlp_grid_search.best_estimator_
    save_model(best_mlp, "best_mlp.pkl")


In [ ]:
print("RF: ",rf_grid_search.best_params_)
print("XBG: ",xgb_grid_search.best_params_)
print("MLP: ",mlp_grid_search.best_params_)

### 10- Model Evaluation
***

In [ ]:
def evaluate_model(model, model_name):
    print(model_name)
    y_pred = model.predict(X_test)
    
    print("Test Accuracy:")
    print(accuracy_score(y_test, y_pred))
    
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=le_target.classes_))
    
    print("Confussion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=le_target.classes_, yticklabels=le_target.classes_) #type:ignore
    plt.show()

In [ ]:
evaluate_model(best_random_forest, "Random Forest")

In [ ]:
evaluate_model(best_xgb_boost, "XGBoost")

In [ ]:
evaluate_model(best_mlp, "Neural Network")

### 11- Ensemble Learning
***

In [ ]:
model_list = [
    best_random_forest,
    best_xgb_boost,
    best_mlp,
]

preds_all = np.array([model.predict(X_test) for model in model_list]) #type:ignore

hard_preds = stats.mode(preds_all, axis=0, keepdims=False)[0]

hard_acc = accuracy_score(y_test, hard_preds)
hard_acc